In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import os
import sys
import json
from scipy.stats import variation
from mplcursors import cursor

In [9]:
default_heights = [i for i in range(0,201,20)]

In [13]:
filename = 'data\\2023-08-26 04-52-40.csv'
index_col = 'timestamp'

In [14]:
def get_CSV_file(filename, index_col):
   df = pd.read_csv(filename, index_col=index_col, parse_dates=True)
   return df

In [20]:
df = get_CSV_file(filename, index_col)
df

,hum_pressure,hum_temp,hum_rel_hum,hum_hum_temp,hum_time,hum_longitude,hum_latitude,hum_altitude,hum_sat,pom_ozone,pom_cell_temp,pom_cell_pressure,pom_photo_volt,pom_power_supply,pom_latitude,pom_longitude,pom_altitude
timestamp,,,,,,,,,,,,,,,,,
2023-08-26 04:52:40,100560.0,2561.0,730.0,2611.0,-999.0,0.0,0.0,-17000.0,0.0,14.0,309.1,739.5,1.0117,11.9,0.0,0.0,0.0
2023-08-26 04:52:41,100562.0,2552.0,730.0,2611.0,-999.0,0.0,0.0,-17000.0,0.0,14.0,309.1,739.5,1.0117,11.9,0.0,0.0,0.0
2023-08-26 04:52:42,100559.0,2555.0,730.0,2612.0,-999.0,0.0,0.0,-17000.0,0.0,14.0,309.1,739.5,1.0117,11.9,0.0,0.0,0.0
2023-08-26 04:52:43,100560.0,2552.0,729.0,2612.0,-999.0,0.0,0.0,-17000.0,0.0,14.0,309.1,739.5,1.0117,11.9,0.0,0.0,0.0
2023-08-26 04:52:44,100560.0,2555.0,729.0,2614.0,-999.0,0.0,0.0,-17000.0,0.0,14.0,309.1,739.5,1.0117,11.9,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-26 05:46:28,100572.0,2527.0,756.0,2611.0,-999.0,348114580.0,319103006.0,47060.0,9.0,27.1,308.9,739.1,0.9987,12.0,0.0,0.0,99.9
2023-08-26 05:46:29,100578.0,2529.0,756.0,2612.0,-999.0,348114548.0,319102968.0,45578.0,9.0,27.1,308.9,739.1,0.9987,12.0,0.0,0.0,99.9
2023-08-26 05:46:30,100576.0,2533.0,756.0,2611.0,-999.0,348114540.0,319102951.0,43930.0,9.0,27.1,308.9,739.1,0.9987,12.0,0.0,0.0,99.9


In [62]:
col = df['hum_pressure'].to_numpy().T
windows = np.lib.stride_tricks.sliding_window_view(col, window_shape=10)
vars_are_small = np.where([np.var(window)<10 for window in windows],1,0)
# vars_are_small = np.where(vars_are_small.all(1)==1,1,0) # merge the column conditions
# d = np.diff(vars_are_small)
# d
vars_are_small

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
def get_idxs_of_low_variance(df, step_cols, var_ths, window_size):
    col = df[step_cols].to_numpy().T
    # a.shape=[n_cols,T], var_ths.shape=[n_cols] or float, window_size=[n_cols, window]
    window = np.lib.stride_tricks.sliding_window_view(col, window_shape=window_size)
    vars_are_small = np.where(stats.variance(window)<var_ths, 1, 0)
    vars_are_small = np.where(vars_are_small.all(1)==1,1,0) # merge the column conditions
    d = np.diff(vars_are_small)
    starts = np.argwhere(d==1)[:,0]+1
    ends = np.argwhere(d==-1)[:,0]+window_size[-1]-1
    if starts[0]>ends[0]: starts = np.concatenate([np.array([0]),starts])
    if starts[-1]>ends[-1]: ends = np.concatenate([ends,np.array([a.shape[-1]-1])])
    step_idxs = [np.arange(s,e+1) for s,e in zip(starts,ends)]
    return starts, ends, step_idxs

In [2]:
def get_idxs_of_low_variance(a, var_ths, window_size):
    # a.shape=[n_cols,T], var_ths.shape=[n_cols] or float, window_size=[n_cols, window]
    a_windows = np.lib.stride_tricks.sliding_window_view(a,window_shape=window_size)
    vars_are_small = (np.where(a_windows.var(-1)<var_ths,1,0))[0]
    vars_are_small = np.where(vars_are_small.all(1)==1,1,0) # merge the column conditions
    d = np.diff(vars_are_small)
    starts = np.argwhere(d==1)[:,0]+1
    ends = np.argwhere(d==-1)[:,0]+window_size[-1]-1
    if starts[0]>ends[0]: starts = np.concatenate([np.array([0]),starts])
    if starts[-1]>ends[-1]: ends = np.concatenate([ends,np.array([a.shape[-1]-1])])
    step_idxs = [np.arange(s,e+1) for s,e in zip(starts,ends)]
    return starts, ends, step_idxs

In [ ]:
def plot_f_with_selections(df, cols, selections, idxs_to_plot=None, x_is_time=False, invert_y=False, to_use_cursor=True):
    if idxs_to_plot is None: idxs_to_plot= np.arange(len(df))
    fig, ax = plt.subplots(figsize=(10,4))
    if x_is_time:
        x = df.index[idxs_to_plot]
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M, %d/%m'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=-45, ha="left")
    else:
        x = np.arange(idxs_to_plot.shape[0])[idxs_to_plot]
    for col in cols:
        ax.plot(x, df[col].to_numpy()[idxs_to_plot])
    if invert_y: plt.gca().invert_yaxis()
    for i,selection in enumerate(selections):
        ax.axvspan(x[selection[0]], x[selection[1]], alpha=0.2, color='green')
        ax.text(x[selection[0]],ax.get_ylim()[1]*0.8+ax.get_ylim()[0]*0.2, f"{i}",fontsize=10, rotation=0)
    if to_use_cursor: cursor(hover=True)
    plt.show()